In [1]:
!pip install -Uq bitsandbytes
!pip install -Uq accelerate optimum
!pip install -Uq transformers
!pip install -Uq peft
!pip install -Uq trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [1]:
import torch
import bitsandbytes
#import accelerate
import transformers
import optimum
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments

In [2]:
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_kbit_training,
)
from trl import DPOTrainer
import pandas as pd
from datasets import load_dataset
import os

In [3]:
# Comment/Uncomment and use as per wish
# MODEL_PATH = "/kaggle/input/gemma/pytorch/2b-it/2" # "/kaggle/input/gemma/pytorch/2b-it/2" "/kaggle/input/gemma/transformers/2b-it/2"
# MODEL_PATH = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

MODEL_PATH = "/kaggle/input/phi/transformers/2/1"


torch.set_default_device("cuda")

#model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# load the training dataset
#dataset = load_dataset("json", data_files={'train': dataset_file})
dataset = pd.read_csv("/content/dpo_dataset_v1.csv")
#dataset = dataset['train'].shuffle(seed=42)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = "auto"

base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
)
base_model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [6]:
dataset = pd.read_csv("/content/dpo_dataset_v1.csv", index_col=0)
dataset.head()

,original_text,rewritten_text,chosen_prompt,rejected_prompt,chosen_score,rejected_score
0,The task you have given me is quite complex an...,## Analysis of the Text\n\nThis text explores ...,Engage in a thoughtful exploration of the cont...,## Most Accurate Prompt:\n\nWrite an analysis ...,1,0
1,The philosophy of happiness is a branch of phi...,"## Analysis of the Text: ""The Philosophy of Ha...",Immerse yourself in a detailed analysis of the...,## Prompt used to convert the original text to...,1,0
2,The impact of technology on the socio-cultural...,## The Whimsical Text with Special Characters ...,Add a touch of whimsy to the following text by...,## Most Accurate Prompt:\n\nWrite a transforme...,1,0
3,Title: Debating the Moral and Legal Complexiti...,"**Summary:**\n\nCognitive dissonance, a psycho...",Capture the essence of the following text in a...,The prompt used to convert the original text t...,1,0
4,Predicting the future of social justice in mod...,## Analysis of the Text\n\nThis text provides ...,Navigate through the maze of contrasting opini...,"**Prompt:** Write an analysis of the text ""Pre...",1,0


In [7]:
output_dir = "/content/"

In [8]:
# Assuming `df` is your DataFrame
dataset.to_json("your_dataset.json", orient="records", lines=True)

# Load the training dataset
dataset_file = "your_dataset.json"
dataset = load_dataset("json", data_files={'train': dataset_file})
dataset = dataset['train'].shuffle(seed=42)

def truncate_text(example, max_length=900):
    # Tokenize the original and rewritten texts to check their length
    tokens_original = tokenizer.encode(example['original_text'], add_special_tokens=False)
    tokens_rewritten = tokenizer.encode(example['rewritten_text'], add_special_tokens=False)
    #print(len(tokens_original), len(tokens_rewritten))

    # Check if the length exceeds max_length and truncate if necessary
    if len(tokens_original) > max_length:
        # Decode back to text after truncating
        example['original_text'] = tokenizer.decode(tokens_original[:max_length], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    if len(tokens_rewritten) > max_length:
        # Decode back to text after truncating
        example['rewritten_text'] = tokenizer.decode(tokens_rewritten[:max_length], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return example


def get_prompt(example):
    # Assuming `tokenizer` and other necessary components are defined elsewhere in your notebook
    example = truncate_text(example)
    #og_text = truncate_text(example['original_text'])
    #rewritten_text = truncate_text(example['rewritten_text'])

    prompt_sample = [
        {"role": "system", "content": "From the given original and rewritten texts, predict the rewrite prompt used to transform the original text."},
        {"role": "user", "content": f"Original: {example['original_text']} ----- Rewritten: {example['rewritten_text']} "}
    ]
    prompt_for_model = tokenizer.apply_chat_template(prompt_sample, tokenize=False)
    example['prompt'] = prompt_for_model

    example['chosen'] = example['chosen_prompt'] + tokenizer.eos_token
    example['rejected'] = example['rejected_prompt'] + tokenizer.eos_token

    return example



Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
# Map the function over the dataset
dataset = dataset.map(get_prompt)
dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



Dataset({
    features: ['original_text', 'rewritten_text', 'chosen_prompt', 'rejected_prompt', 'chosen_score', 'rejected_score', 'prompt', 'chosen', 'rejected'],
    num_rows: 1000
})

In [10]:
dataset = dataset.rename_column("chosen_score", "score_chosen")
dataset = dataset.rename_column("rejected_score", "score_rejected")

In [11]:
example = dataset[0]
example

{'original_text': "The geopolitical tensions in the Middle East have had significant socio-cultural impacts and far-reaching global ramifications. The region is a hotbed of conflict, with ongoing struggles for power, resources, and political influence. These tensions have resulted in violence, displacement, and instability, which have had profound effects on the people of the region and beyond.\n\nOne of the most significant socio-cultural impacts of these tensions is the displacement of millions of people. The conflicts in Syria, Iraq, and Yemen have forced millions of people to flee their homes, creating a massive refugee crisis. Many of these refugees have sought safety in neighboring countries, while others have made the dangerous journey to Europe. This mass displacement has put a significant strain on resources and social services in host countries and has led to cultural and social tensions.\n\nThe conflicts in the Middle East have also had a profound impact on the region's cult

In [12]:
# Define the system message
input_text = example["prompt"]

# Tokenize input text, ensuring to generate an attention mask this time
inputs = tokenizer(input_text, return_tensors="pt", max_length=2000, truncation=True, padding=True)

# For open-ended generation, setting pad_token_id explicitly if your model does not have one set
if base_model.config.pad_token_id is None:
    base_model.config.pad_token_id = base_model.config.eos_token_id

# Generate output using the updated inputs
outputs = base_model.generate(**inputs, max_length=2000, pad_token_id=base_model.config.pad_token_id)
text = tokenizer.batch_decode(outputs)[0]

# Output the model's generated text and the actual rewrite prompt for comparison
actual_rewrite_prompt = example['chosen_prompt']
print("Generated Prompt:", text)
print("Actual Rewrite Prompt:", actual_rewrite_prompt)


Generated Prompt: <|im_start|>system
From the given original and rewritten texts, predict the rewrite prompt used to transform the original text.<|im_end|>
<|im_start|>user
Original: The geopolitical tensions in the Middle East have had significant socio-cultural impacts and far-reaching global ramifications. The region is a hotbed of conflict, with ongoing struggles for power, resources, and political influence. These tensions have resulted in violence, displacement, and instability, which have had profound effects on the people of the region and beyond.

One of the most significant socio-cultural impacts of these tensions is the displacement of millions of people. The conflicts in Syria, Iraq, and Yemen have forced millions of people to flee their homes, creating a massive refugee crisis. Many of these refugees have sought safety in neighboring countries, while others have made the dangerous journey to Europe. This mass displacement has put a significant strain on resources and socia

In [13]:
base_model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [14]:
# from https://github.com/mlabonne/llm-course/blob/main/Fine_tune_a_Mistral_7b_model_with_DPO.ipynb
lora_dropout=0.05
lora_alpha=16
lora_r=16
learning_rate=5e-5

batch_size = 4

def create_peft_config(model):
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        lora_dropout=lora_dropout,
        lora_alpha=lora_alpha,
        r=lora_r,
        bias="none",
        target_modules = ["q_proj", "k_proj", "v_proj"] #, "o_proj", "gate_proj"]
    )

    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)

    model.print_trainable_parameters()

    return model, peft_config

model, lora_config = create_peft_config(base_model)

trainable params: 7,864,320 || all params: 2,787,548,160 || trainable%: 0.2821231974697076


In [15]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    learning_rate=learning_rate,

    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    warmup_steps=50,
    logging_steps=1,
    num_train_epochs=1,
    save_steps=50,
    lr_scheduler_type="cosine",
    optim="paged_adamw_32bit",
)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [17]:
config_content = """
compute_environment: LOCAL_MACHINE
deepspeed_config: {}
distributed_type: NO
fsdp_config: {}
machine_rank: 0
main_process_ip: null
main_process_port: null
main_training_function: main
mixed_precision: no
num_machines: 1
num_processes: 1
use_cpu: false
"""

config_path = "/content/my_accelerate_config.yaml"
with open(config_path, "w") as config_file:
    config_file.write(config_content)


In [18]:
#!accelerate launch --config_file /content/my_accelerate_config.yaml

In [19]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    learning_rate=learning_rate,

    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    warmup_steps=50,
    logging_steps=1,
    num_train_epochs=1,
    save_steps=50,
    lr_scheduler_type="cosine",
    optim="paged_adamw_32bit",
)

trainer = DPOTrainer(
    base_model, # model base_model
    ref_model=None,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=lora_config,
    beta=0.1,
    max_prompt_length=2048, #changed from 1024
    max_length=1536,
)

trainer.train()

# todo: during training getting these warning:

# i guess this is on the base model, need to check. in that case this is fine
# UserWarning: None of the inputs have requires_grad=True. Gradients will be None

# seems that this can be ignored:
# Could not estimate the number of tokens of the input, floating-point operations will not be computed

output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


RuntimeError: Expected a 'cuda' device type for generator but found 'cpu'

In [23]:
dir(trainer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activate_neftune',
 '_add_sm_patterns_to_gitignore',
 '_created_lr_scheduler',
 '_deactivate_neftune',
 '_finish_current_push',
 '_fsdp_qlora_plugin_updates',
 '_gather_and_numpify',
 '_get_collator_with_removed_columns',
 '_get_eval_sampler',
 '_get_learning_rate',
 '_get_output_dir',
 '_get_train_sampler',
 '_globalstep_last_logged',
 '_hp_search_setup',
 '_inner_training_loop',
 '_issue_warnings_after_load',
 '_load_best_model',
 '_load_from_checkpoint',
 '_load_optimizer_and_scheduler',
 '_load_rng_state',
 '_loggers_initialized',
 '_maybe_log_save_evaluate',
 '_memory_tracker',
 '_move_model_to_de

In [25]:
dir(trainer.accelerator)

['__class__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_custom_objects',
 '_dataloaders',
 '_do_sync',
 '_get_devices',
 '_get_named_parameters',
 '_load_model_state_pre_hook',
 '_models',
 '_optimizers',
 '_prepare_deepspeed',
 '_prepare_ipex',
 '_prepare_megatron_lm',
 '_prepare_msamp',
 '_prepare_one',
 '_save_model_state_pre_hook',
 '_schedulers',
 'accumulate',
 'autocast',
 'autocast_handler',
 'backward',
 'check_trigger',
 'clear',
 'clip_grad_norm_',
 'clip_grad_value_',
 'dataloader_config',
 'ddp_handler',
 'deepspeed_engine_wrapped',
 'device',
 'device_placement',
 'dispatch_batches',
 'distributed_type',
 'end_training',
 'even_batches',
 'flag